In [20]:
#from PIL import Image, ImageDraw
import numpy as np
#import matplotlib.pyplot as plt
import math
#%matplotlib inline
import numpy as np
import concurrent.futures

### Grid map representation 

Square grid map class represents agent's environment

- width -- the number of columns in grid
- height -- the number of rows in grid
- cells -- the binary matrix, which represents the grid. 0 - cell is traversable
1 - cell is blocked

In [21]:
class Map:

    # Default constructor
    def __init__(self):
        self.width = 0
        self.height = 0
        self.cells = []
    
    # Converting a string (with '#' representing obstacles and '.' representing free cells) to a grid
    def ReadFromString(self, cellStr, width, height):
        self.width = width
        self.height = height
        self.cells = [[0 for _ in range(width)] for _ in range(height)]
        cellLines = cellStr.split("\n")
        i = 0
        j = 0
        for l in cellLines:
            if len(l) != 0:
                j = 0
                for c in l:
                    if c == '.':
                        self.cells[i][j] = 0
                    elif c == '@':
                        self.cells[i][j] = 1
                    else:
                        continue
                    j += 1
                # TODO
                if j != width:
                    raise Exception("Size Error. Map width = ", j, ", but must be", width )
                
                i += 1

        if i != height:
            raise Exception("Size Error. Map height = ", i, ", but must be", height )
    
    # Initialization of map by list of cells.
    def SetGridCells(self, width, height, gridCells):
        self.width = width
        self.height = height
        self.cells = gridCells

    # Check if the cell is on a grid.
    def inBounds(self, i, j):
        return (0 <= j < self.width) and (0 <= i < self.height)
    
    # Check if thec cell is not an obstacle.
    def Traversable(self, i, j):
        return not self.cells[i][j]

    # Get a list of neighbouring cells as (i,j) tuples.
    # It's assumed that grid is 4-connected (i.e. only moves into cardinal directions are allowed)
    def GetNeighbors(self, i, j):
        neighbors = []
        delta = [[0, 1], [1, 0], [0, -1], [-1, 0]]

        for d in delta:
            if self.inBounds(i + d[0], j + d[1]) and self.Traversable(i + d[0], j + d[1]):
                neighbors.append((i + d[0], j + d[1]))

        return neighbors
    
    def get_item(self, i, j):
        
        return self.cells[i][j]

Computes a cost of transition from cell `(i1, j1)` to cell `(i2, j2)`. In case of 4-connected grid it is always equal to 1. However if the connectivity of a grid exceeds 4 the cost of a transition may vary.

In [22]:
def CalculateCost(i1, j1, i2, j2):
    return math.sqrt((i1 - i2) ** 2 + (j1 - j2) ** 2)

### Search Node Representation

Node class represents a search node

- i, j: coordinates of corresponding grid element
- g: g-value of the node
- h: h-value of the node
- F: f-value of the node
- parent: pointer to the parent-node 

In [23]:
class Node:
    def __init__(self, i, j, g = math.inf, h = math.inf, F = None, parent = None, alternating=None):
        self.i = i
        self.j = j
        self.g = g
        if F is None:
            self.F = self.g + h
        else:
            self.F = F 
        # Calculation for avoiding of reexpansions
        if alternating is not None:
            self.F = max(self.F, 2 * self.g)
        self.parent = parent
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j)
    
    def __hash__(self):
        
        return self.i << 16 | self.j
        #return self.i + self.j
        

Auxiliary function that visualizes obstacles, start and goal cells, a path and OPEN and CLOSED.

In [24]:
def Draw(gridMap : Map, start : Node = None, goal : Node = None, path : list = None, nodesExpanded = None, nodesOpened = None):
    k = 5
    hIm = gridMap.height * k
    wIm = gridMap.width * k
    im = Image.new('RGB', (wIm, hIm), color = 'white')
    draw = ImageDraw.Draw(im)
    for i in range(gridMap.height):
        for j in range(gridMap.width):
            if(gridMap.cells[i][j] == 1):
                draw.rectangle((j * k, i * k, (j + 1) * k - 1, (i + 1) * k - 1), fill=( 70, 80, 80 ))

    if nodesOpened is not None:
        for node in nodesOpened:
            draw.rectangle((node.j * k, node.i * k, (node.j + 1) * k - 1, (node.i + 1) * k - 1), fill=(213, 219, 219), width=0)

    if nodesExpanded is not None:
        for node in nodesExpanded:
            draw.rectangle((node.j * k, node.i * k, (node.j + 1) * k - 1, (node.i + 1) * k - 1), fill=( 131, 145, 146 ), width=0)

    if path is not None:
        #print('Path drawing')
        #print(len(path))
        for step in path:
            #print('STEP drawing')
            if (step is not None):
                #print('Step not None')
                if (gridMap.Traversable(step.i, step.j)):
                    #print('Traversable')
                    draw.rectangle((step.j * k, step.i * k, (step.j + 1) * k - 1, (step.i + 1) * k - 1), fill=(52, 152, 219), width=0)
                    #pass
                else:
                    #pass
                    #print('Not Traversable')
                    draw.rectangle((step.j * k, step.i * k, (step.j + 1) * k - 1, (step.i + 1) * k - 1), fill=(230, 126, 34), width=0)

    if (start is not None) and (gridMap.Traversable(start.i, start.j)):
        draw.rectangle((start.j * k, start.i * k, (start.j + 1) * k - 1, (start.i + 1) * k - 1), fill=(40, 180, 99), width=0)
    
    if (goal is not None) and (gridMap.Traversable(goal.i, goal.j)):
        draw.rectangle((goal.j * k, goal.i * k, (goal.j + 1) * k - 1, (goal.i + 1) * k - 1), fill=(231, 76, 60), width=0)


    fig, ax = plt.subplots(dpi=150)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.imshow(np.asarray(im))
    
    


### Implementing OPEN and CLOSED
Efficient implementation of OPEN and CLOSED is crucial for any search algorithm. Below you may find basic implementations that use lists to store the elements of OPEN and CLOSED. This is not very efficient (especially for CLOSED). Your task is to create your own implementations.

In [25]:
class OpenList():
    def __init__(self):
        self.elements = []
    
    def __iter__(self):
        return iter(self.elements)

    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True
    

    def GetBestNode(self):
        bestF = math.inf
        bestCoord = 0
        for i in range(len(self.elements)):
            if self.elements[i].F - bestF < -0.000005:
                bestCoord = i
                bestF = self.elements[i].F
                
        best = self.elements.pop(bestCoord)
        return best
    #Create your own modification of CLOSED#
    

    def AddNode(self, item : Node):
        for coord in range(len(self.elements)):
            if self.elements[coord].i == item.i and self.elements[coord].j == item.j:
                if self.elements[coord].g - item.g > 0.000005:
                    self.elements[coord].F = item.F
                    self.elements[coord].g = item.g
                    self.elements[coord].parent = item.parent
                    return
                else:
                    return
        self.elements.append(item)
        return
    
    def WasOpened(self, item : Node):
        return item in self.elements
    
    def GetNode(self, item : Node):
        i = self.elements.index(item)
        return self.elements[i]
    
    def GetBestg(self):
        bestF = math.inf
        for i in range(len(self.elements)):
            if self.elements[i].F - bestF < -0.000005:
                bestF = self.elements[i].F
                
        return bestF
    
    def Connect(self, item):
        self.elements += item.elements
        return self


In [26]:
class YourClosed():

    def __init__(self):
        self.elements = set()


    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)
    
    # AddNode is the method that inserts the node to CLOSED
    def AddNode(self, item : Node, *args):
        self.elements.add(item)

    # WasExpanded is the method that checks if a node has been expanded
    def WasExpanded(self, item : Node, *args):
        return item in self.elements
    
    
    def Connect(self, item):
        self.elements = self.elements.union(item.elements)
        return self
    
    def GetNode(self, item : Node):
        #return self.elements.intersection({item}).pop()
        #q = {item}.intersection(self.elements).pop()
        #print('fdfd', q is item)
        for q in self.elements:
            if item == q:
                #print('eq', item is q)
                return q
        #return {item}.intersection(self.elements).pop()
        

### Validating the results

MakePath is function that unwinds the path using the parent pointers and returns both a path and its length.

In [27]:
# Creating a path by tracing parent pointers from the goal node to the start node
# It also returns path's length.
def MakePath(goal):
    length = goal.g
    current = goal
    path = []
    while current.parent:
        path.append(current)
        current = current.parent
    path.append(current)
    return path[::-1], length

# Generating goal states
There are main function for data generating "generate" and auxiliary functions

In [28]:
from random import randint

def write_file(target_map, save_path):
    
    with open(save_path, 'w') as fopen:
        for line in target_map:
            str_line = ''.join([str(element) + ' ' for element in line]) + '\n'
            fopen.write(str_line)
            
    return

def generate_one_point(taskMap, Goals):
    
    
    height = taskMap.height
    width = taskMap.width
        
    iGoal = randint(0, height)
    jGoal = randint(0, height)

    while (iGoal, jGoal) in Goals:
        iGoal = randint(0, height)
        jGoal = randint(0, width)
    
    Goals.add((iGoal, jGoal))
    
    return iGoal, jGoal

from copy import deepcopy

def get_target_map(taskMap, CLOSED):
    
    height = taskMap.height
    width = taskMap.width
    
    target_map = [[0 for j in range(width)] for i in range(height)]
    
    for i in range(height):
        for j in range(width):
            if taskMap.cells[i][j] == 1:
                target_map[i][j] = 10 ** 6
                
    for node in CLOSED:
        i = node.i
        j = node.j
        target_map[i][j] = node.g
        
    return target_map

def get_Goal_map(taskMap, iGoal, jGoal):
    
    height = taskMap.height
    width = taskMap.width
    
    Goal_map = [[0 for j in range(width)] for i in range(height)]
    Goal_map[iGoal][jGoal] = 1
    
    return Goal_map

    

def generate(args):
    
    #print('Start generating')
    
    taskMap, input_directory, output_directory, map_file, SearchFunction, quantity_Goals = args
    
    input_path1 = input_directory + 'input_map/' + map_file
    write_file(taskMap.cells, input_path1)
        
    Goals = set()
    limit = taskMap.height * taskMap.width / 4
    
    for i in range(quantity_Goals):

        closed_size = 0
        
        start = time.time()
        j = 0
        while closed_size < limit:
            #print(i, j, end='\r')
            
            iGoal, jGoal = generate_one_point(taskMap, Goals)
            CLOSED = SearchFunction(taskMap, iGoal, jGoal)
            closed_size = len(CLOSED)
            #_ = input()
            
            j += 1
            
        
        input_path2 = input_directory + 'Goal_map/' + str(iGoal) + '_' + str(jGoal) + '_' + map_file
        Goal_map = get_Goal_map(taskMap, iGoal, jGoal)
        write_file(Goal_map, input_path2)
        
        target_map = get_target_map(taskMap, CLOSED)
        output_path = output_directory + str(iGoal) + '_' + str(jGoal) + '_' + map_file
        write_file(target_map, output_path)
        
        #o_map = np.array(target_map)
        #img = Image.fromarray(o_map)
        #img.show()
        
        #print('time = {}'.format(time.time() - start), end='\r')
        
        
    return
        

### Read task and map

In [29]:
def ReadMapFromFile(path):
    
    tasksFile = open(path)
    count = 0
    _ = tasksFile.readline()
    height = int(tasksFile.readline().split()[1])
    width = int(tasksFile.readline().split()[1])
    _ = tasksFile.readline()
    cells = [[0 for _ in range(width)] for _ in range(height)]
    i = 0
    j = 0

    allowed_quantity = 0
    not_allowed_quantity = 0
    
    for l in tasksFile:
        j = 0
        for c in l:
#             print(c)
#             _ = input()
            if c == '.':
                cells[i][j] = 0
                allowed_quantity += 1
            elif c == '@':
                cells[i][j] = 1
                not_allowed_quantity += 1
            else:
                continue
            
            j += 1
            
        if j != width:
            raise Exception("Size Error. Map width = ", j, ", but must be", width, "(map line: ", i, ")")
                
        i += 1
        if(i == height):
            break
    
    return (width, height, cells)

def ReadTaskFromFile(path):
    
    fopen = open(path)
    _ = fopen.readline()
    
    for i, line in enumerate(fopen):
#         if i > 300:
#             break
        split_line = line.split()
        #print(split_line)
        yield [int(x) for x in split_line[4:-1]] + [float(split_line[-1])]

### OneTest

OneTest runs `SearchFunction` on set of differnt tasks (from directory `Data/`) with `*args` as optional arguments and for every task displays one of these short reports:

- 'Path found!' and some statistics -- path was found
- 'Path not found!' -- path was not found 
- 'Execution error' -- an error occurred while executing the `SearchFunction` 
In first two cases function also draws visualisation of the task. 

Onetest return a dictionary with statistics of path finding. Dictionary contains next fields:
- correct -- the correctness of path length (`True/False`). It is not necessary because real path value is calculated for actions which include diagonal move so it is not True for our case 
- path_len -- the length of every path (0.0 if path not found)
- nc_len -- the number of created nodes task
- st_len -- the number of steps of algorithm task

In [1]:
def OneTest(args):
    
    taskMap, iStart, jStart, iGoal, jGoal, length, SearchFunction, heuristicFunction = args
    
    start_t = time.time()
    result = SearchFunction(taskMap, iStart, jStart, iGoal, jGoal, heuristicFunction)
    t = time.time() - start_t

    nodesExpanded = result[2]
    nodesOpened = result[3]
    h_start = result[4]

    nc_len = len(nodesOpened) + len(nodesExpanded)
    st_len = len(nodesExpanded)

    if result[0]:
        path = MakePath(result[1]) 
        path_len = path[1]
        correct = (path_len == length)
#         print("Path found! : " + "Length: " + str(path_len) + ". Nodes created: " + \
#               str(nc_len) + ". Number of steps: " \
#               + str(st_len) + ". Correct: " + str(correct), end='\r')
        #print('Time = {}'.format(t))
        #Draw(taskMap, Node(iStart, jStart), Node(iGoal, jGoal), path[0], nodesExpanded, nodesOpened)
    else:
        print("Path not found! iStart = {} jStart = {} iGoal = {} jGoal = {}".format(iStart, jStart, iGoal, jGoal))
        correct = False
        path_len = None



    return correct, path_len, nc_len, st_len, result[0], t, h_start

### Functions for experiments
- "SeveralTests" is parallelling Onetest for several task. For paralleling generator "arg_generator" is used to generate arguments for function
- "MassiveTest" is experiment running function without parallelling

In [149]:
import time
import os

def arg_generator(SearchFunction, GetheuristicFunction, size):
    maps_directory = 'data/files/maps/' + str(size) + '/'
    scen_directory = 'data/files/scens/' + str(size) + '/'
    
    map_list = os.listdir(maps_directory)
    map_list.sort()
    map_list = map_list
    
    task_file_list = os.listdir(scen_directory)
    task_file_list.sort()
    task_file_list = task_file_list
    
    for map_file, task_file in zip(map_list, task_file_list):
        
        taskFileName = maps_directory + map_file
        task_file_path = scen_directory + task_file
        
        #print(taskFileName)
        width, height, cells = ReadMapFromFile(taskFileName)
        taskMap = Map()
        taskMap.SetGridCells(width,height,cells)
        
        #print(task_file_path)
        for task in ReadTaskFromFile(task_file_path):
            jStart, iStart, jGoal, iGoal, length = task 
            #print(iGoal, jGoal)
            heuristicFunction = GetheuristicFunction(iGoal=iGoal, jGoal=jGoal, taskMap=taskMap)
            #print('arg generate')
            yield (taskMap, iStart, jStart, iGoal, jGoal, length, SearchFunction, heuristicFunction)
    

def SeveralTests(SearchFunction, GetheuristicFunction, size):
    
    stat = dict()
    stat["corr"] = []
    stat["len"] = []
    stat["nc"] = []
    stat["st"] = []
    stat["h"] = []
        
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        for results in executor.map(OneTest, arg_generator(SearchFunction, GetheuristicFunction, size)):
            correct, path_len, nc_len, st_len, found, t, h_start = results
            if found:
                stat["corr"].append(correct)
                stat["len"].append(path_len)
                stat["nc"].append(nc_len)
                stat["st"].append(st_len)
                stat["h"].append(h_start)
            else:
                pass
        
    return stat


def MassiveTest(SearchFunction, GetheuristicFunction, size, model=''):
    
    model_name = 'model_' + str(size) + '_' + model + '.pt'
    
    maps_directory = 'data/files/maps/' + str(size) + '/'
    scen_directory = 'data/files/scens/' + str(size) + '/'
    
    stat = dict()
    stat["corr"] = []
    stat["len"] = []
    stat["nc"] = []
    stat["st"] = []
    stat["h"] = []
        
    
    map_list = os.listdir(maps_directory)
    map_list.sort()
    map_list = map_list
    
    task_file_list = os.listdir(scen_directory)
    task_file_list.sort()
    task_file_list = task_file_list
    
    for map_file, task_file in zip(map_list, task_file_list):
        
        taskFileName = maps_directory + map_file
        task_file_path = scen_directory + task_file
        
        #print(map_file)
        width, height, cells = ReadMapFromFile(taskFileName)
        taskMap = Map()
        taskMap.SetGridCells(width,height,cells)
        
        #print(task_file)
        for it, task in enumerate(ReadTaskFromFile(task_file_path)):
            jStart, iStart, jGoal, iGoal, length = task 
            print('iter = {}'.format(it), end='\r')
            heuristicFunction = GetheuristicFunction(iGoal=iGoal, jGoal=jGoal, taskMap=taskMap, model=model_name)
            #heuristicFunction = GetheuristicFunction
            #print('arg generate')
            args = (taskMap, iStart, jStart, iGoal, jGoal, length, SearchFunction, heuristicFunction)
            correct_len, path_len, nc_len, st_len, found, t, h_start = OneTest(args)

            if found:
                stat["corr"].append(correct_len)
                stat["len"].append(path_len)
                stat["nc"].append(nc_len)
                stat["st"].append(st_len)
                stat["h"].append(h_start)
            else:
                pass
        
    return stat
        

### A* algorithm 
- You will have cases when path will be not found for map size 64 and 128. Don't worry. It is normal because they were created from original map using resizing.

In [112]:
def AStar(gridMap : Map, iStart : int, jStart : int, iGoal : int, jGoal : int, \
          heuristicFunction, openType = OpenList, closedType = YourClosed):
    #print('AStart Start')
    h_start = heuristicFunction.GetValue(iStart, jStart+ str(size) + '_zero.pt')
    #h_start = heuristicFunction(iStart, jStart, iGoal, jGoal)
    #print(heuristicFunction.iGoal, heuristicFunction.jGoal)
    #print(iStart, jStart)
    #print(h_start)
    #_ = input()
#     print(gridMap.GetNeighbors(iStart, jStart), gridMap.GetNeighbors(iGoal, jGoal), \
#           gridMap.cells[iStart][jStart], gridMap.cells[iGoal][jGoal])
    Start = Node(iStart, jStart, g=0, h=h_start)
    Goal = Node(iGoal, jGoal)
    
    
    OPEN = openType()
    #print(OPEN.isEmpty())
    OPEN.AddNode(Start)
    CLOSED = closedType()
    
    #print('part1')
    while not OPEN.isEmpty():
        #print('\nwhile')
        
        s = OPEN.GetBestNode()
        #print('got best s')
        CLOSED.AddNode(s)
        #print('addNode')
    
        if s == Goal:
            #print('fGOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
            return True, s, CLOSED, OPEN, h_start
        
        #print('get neighbors')stat["corr"].append(correct)
        else:
            pass
        neighbors = gridMap.GetNeighbors(s.i, s.j)
        #print('got neighbors')
        for new_s in neighbors:
            
            new_i, new_j = new_s
            h = heuristicFunction.GetValue(new_i, new_j)
            #h = heuristicFunction(new_i, new_j, iGoal, jGoal)
            new_g = s.g + CalculateCost(s.i, s.j, new_i, new_j)
            new_node = Node(new_i, new_j, new_g, h=h, parent=s)
            
            if not CLOSED.WasExpanded(new_node):
                
                OPEN.AddNode(new_node)
                
                
    return False, None, CLOSED, OPEN, h_start

### ManhattanDistance

In [55]:
import math

class ManhattanDistance():

    def __init__(self, **kwargs):
        
        self.iGoal = kwargs['iGoal']
        self.jGoal = kwargs['jGoal']
        
    def GetValue(self, i1, j1):

        return math.fabs(i1 - self.iGoal) + math.fabs(j1 - self.jGoal)

# def ManhattanDistance(i1, j1, i2, j2):
#     return math.fabs(i1 - i2) + math.fabs(j1 - j2)

Experiments

In [2]:
# ManStat = SeveralTests(AStar, ManhattanDistance, 64)
ManStat = MassiveTest(AStar, ManhattanDistance, 64)

In [3]:
ManStat_128 = MassiveTest(AStar, ManhattanDistance, 128)

### Learnt heutristuc function
- GetLearntFunction - non-masked learnt heuristic function
- GetLearntFunctionZero - masked learnt heuristic function. In this ipynb it is not implemented

In [150]:
import torch

class GetLearntFunction():
    
    def __init__(self, iGoal, jGoal, taskMap, model):
        
        size = taskMap.height
        path = 'model/' + model
        model = torch.load(path, map_location=torch.device('cpu'))
        model.eval()

        in_data1 = taskMap.cells
        in_data2 = self.get_Goal_map(taskMap, iGoal, jGoal)
        in_data = torch.tensor([[in_data1, in_data2]], dtype=torch.float32)
        
        #print(model(in_data)[0][0])
        with torch.no_grad():
            self.heuristic = model(in_data)[0][0]
    
    def GetValue(self, i, j):
        return self.heuristic[i][j]
    
    def get_Goal_map(self, taskMap, iGoal, jGoal):
    
        height = taskMap.height
        width = taskMap.width

        Goal_map = [[0 for j in range(width)] for i in range(height)]
        Goal_map[iGoal][jGoal] = 1

        return Goal_map
    
    

class GetLearntFunctionZero():
    
    def __init__(self, iGoal, jGoal, taskMap, model):
        
        size = taskMap.height
        path = 'model/model_' + model
        model = torch.load(path, map_location=torch.device('cpu'))
        model.eval()

        in_data1 = taskMap.cells
        in_data2 = self.get_Goal_map(taskMap, iGoal, jGoal)
        in_data = torch.tensor([[in_data1, in_data2]], dtype=torch.float32)
        
        with torch.no_grad():
            self.heuristic = model(in_data)[0][0]
        
    def GetValue(self, i, j):
        return self.heuristic[i][j]
    
    def get_Goal_map(self, taskMap, iGoal, jGoal):
    
        height = taskMap.height
        width = taskMap.width

        Goal_map = [[0 for j in range(width)] for i in range(height)]
        Goal_map[iGoal][jGoal] = 1

        return Goal_map

Experiments

In [4]:
# LearnStat = SeveralTests(AStar, GetLearntFunction, 256)
LearnStat = MassiveTest(AStar, GetLearntFunction, 64)

In [5]:
LearnStat_128 = MassiveTest(AStar, GetLearntFunction, 128)

In [6]:
LearnStat_64_18 = MassiveTest(AStar, GetLearntFunction, 64, model='18')

In [7]:
LearnStat_64_18_mae = MassiveTest(AStar, GetLearntFunction, 64, model='18_mae')

### Analyzing

In [84]:
def analyze(ManStat, LearnStat):
    step = 0.2
    interval = [1 + 0.2 * i for i in range(9)]
    
    quantity = {x : [] for x in interval + [2.8]}
    ratios = {x : 0 for x in interval + [2.8]}
    
    for h_opt, h_base, h_learn, N_base, N_learn in zip(ManStat['len'], ManStat['h'], LearnStat['h'], \
                                                      ManStat['nc'], LearnStat['nc']):
        if h_base == 0:
            continue
            
        difficulty = h_opt / h_base
        for first in interval:
            last = first + step
            
            if first <= difficulty < last:
                quantity[first].append(N_learn / N_base)
        
        if difficulty >= 2.8:
            quantity[2.8].append(N_learn/N_base)
            
    
    for item in quantity:
        if len(quantity[item]) == 0:
            continue
            
        ratios[item] = sum(quantity[item]) / len(quantity[item])
        
    return ratios

### Get ratio 
- ratio - average(N / Nb) for different difficulty parameter
- N - expaned node quantities for learnt heuristic
- Nb - expaned node quantities for Manhattan Distance

In [165]:
def ratios(ManStat, LearnStat):
    ratios = analyze(ManStat, LearnStat)

    print('difficulty| N/Nb')
    print('__________|_____')
    step = 0.2
    for first in ratios:
        last = first + step
        if first == 2.8:
            print('{0:.1f} - inf | {1:.1f}'.format(first, ratios[first]))
        else:
            print('{0:.1f} - {1:.1f} | {2:.1f}'.format(first, last, ratios[first]))
#         print('{0:.1f}'.format(ratios[first]), end=' ')

### Results

In [8]:
ratios(ManStat, LearnStat)

In [9]:
ratios(ManStat, LearnStat_64_18)

In [12]:
ratios(ManStat, LearnStat_64_18_mae)